<a href="https://colab.research.google.com/github/MrBenjaminHolmes/Gravitational-Waves/blob/main/GWpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gravitational Wave Open Data Scripts

##Initialization

In [1]:
!pip install -q gwpy==3.0.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 100.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 45.0.5 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 45.0.5 which is incompatible.


In [ ]:
! pip install -q PyCBC==2.4.1 lalsuite==7.25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 970.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import gwpy
from gwosc.datasets import event_gps
from gwpy.timeseries import TimeSeries
import numpy
import matplotlib.pyplot as plt
from pycbc.waveform import td_approximants, fd_approximants,get_td_waveform

##Noise Comparison 1.1

In [ ]:
#--GW190412 Data-#
gw19 = event_gps('GW190412')
gw19Data = TimeSeries.fetch_open_data('L1', int(gw19)-512, int(gw19)+512, cache=True)
gw19ASD = gw19Data.asd(fftlength=4, method="median")

#--GW150914 Data-#
gw15 = event_gps('GW150914')
gw15Data = TimeSeries.fetch_open_data('L1', int(gw15)-512, int(gw15)+512, cache=True)
gw15ASD = gw15Data.asd(fftlength=4, method="median")

plt.loglog(gw19ASD, label = 'GW190412')
plt.loglog(gw15ASD,label = 'GW150914')
plt.legend()

plt.show()

##Spectogram 1.2

In [ ]:
gps = event_gps("GW170817")
ldata = TimeSeries.fetch_open_data("L1", gps - 300, gps + 300, cache=True)
specgram = ldata.spectrogram2(fftlength=4, overlap=2, window='hann') ** (0.5)
plot = specgram.plot();

ax = plot.gca()
ax.set_yscale('log')
ax.set_ylim(10, 1400)
ax.colorbar(
    clim=(1e-24, 1e-20),
    norm="log",
    label=r"Strain noise [$1/\sqrt{\mathrm{Hz}}$]",
)

##Q-Transform 1.3

In [ ]:
gps = event_gps("GW170814")
segment = (int(gps) - 5, int(gps) + 2)
hData = TimeSeries.fetch_open_data('H1', *segment, verbose=True, cache=True)
lData = TimeSeries.fetch_open_data('L1', *segment, verbose=True, cache=True)
vData = TimeSeries.fetch_open_data('V1', *segment, verbose=True, cache=True)

hq = hData.q_transform(frange=(30, 400), qrange=(4, 20), outseg=(gps-0.2, gps+0.2))
lq = lData.q_transform(frange=(30, 400), qrange=(4, 20), outseg=(gps-0.2, gps+0.2))
vq = vData.q_transform(frange=(30, 400), qrange=(4, 20), outseg=(gps-0.2, gps+0.2))

hq_plot = hq.abs()**0.5
lq_plot = lq.abs()**0.5
vq_plot = vq.abs()**0.5

#-----------Graphing Data-----------#
fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharey=True)

xt_h = hq_plot.times.value - gps
xt_l = lq_plot.times.value - gps
xt_v = vq_plot.times.value - gps

im0 = ax[0].pcolormesh(xt_h, hq_plot.frequencies.value, hq_plot.value.T, vmin=1, vmax=5)
im1 = ax[1].pcolormesh(xt_l, lq_plot.frequencies.value, lq_plot.value.T, vmin=1, vmax=5)
im2 = ax[2].pcolormesh(xt_v, vq_plot.frequencies.value, vq_plot.value.T, vmin=1, vmax=5)

# Axis formatting
ax[0].set_title('H1')
ax[1].set_title('L1')
ax[2].set_title('V1')

ax[1].set_xlabel("Time [s]")

for a in ax:
    a.set_yscale('log')

ax[0].set_ylabel('Frequency [Hz]')


cbar = fig.colorbar(im2, ax=ax[2], pad=0.02)
cbar.set_label('Normalized energy')

fig.subplots_adjust(wspace=0.05)
plt.suptitle("Q-Transforms of GW170814 (H1, L1, V1)", fontsize=14)
plt.show()
#---------------------------------------#

##Strain VS Mass 1.4

In [ ]:
plt.figure(figsize=(10, 6))
for m in [5, 10, 30, 100]:
    hp, hc = get_td_waveform(approximant="IMRPhenomD",
                         mass1=m,
                         mass2=m,
                         delta_t=1.0/4096,
                         f_lower=30)

    plt.plot(hp.sample_times, hp, label='$M_{\odot 1,2}=%s$' % m)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.xlabel('Time (s)')
plt.ylabel('Strain')
plt.show()

##Waveform Generation 1.5
Generating a waveform for the binary neutron star merger GW170817

|  Data  | Value |
| ------------- | ------------- |
| Mass 1  (☉)|  1.46 |
| Mass 2 (☉)|  1.27 |
| Distance (Mpc) |  40 |


In [ ]:
plt.figure(figsize=(10, 6))
hp,hc = get_td_waveform(approximant="IMRPhenomD",
                         mass1=1.46,
                         mass2=1.27,
                         delta_t=1.0/4096,
                         f_lower=30,
                        distance = 40
                        )
plt.plot(hp.sample_times, hp, label='Plus Polarization (hp)')
plt.plot(hc.sample_times, hc, label='Cross Polarization (hc)')
plt.legend()
plt.grid(True)
plt.xlabel('Time (s)')
plt.ylabel('Strain')
plt.show()

plt.plot(hp.sample_times, hp, label='Plus Polarization (hp)')
plt.plot(hc.sample_times, hc, label='Cross Polarization (hc)')
plt.legend()
plt.grid(True)
plt.xlim(-0.05,0)
plt.xlabel('Time (s)')
plt.ylabel('Strain')
plt.show()

In [ ]:
#Define Sample Properties
sample_rate = 1024
data_length = 1024

#Simulate PSD
flow = 10.0
delta_f = 1.0 / 128
flen = int(sample_rate / (2 * delta_f)) + 1
psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

delta_t = 1.0 / sample_rate
ts = pycbc.noise.noise_from_psd(data_length*sample_rate, delta_t, psd, seed=127)

#Inject Waveform into the noise
ts[waveform_start:waveform_start+len(hp1)] += hp1.numpy() * 1E-20

# Prepare PSD for whitening the Noise
delta_f = 1.0 / data_length
flen = int(sample_rate / (2 * delta_f)) + 1
psd_td = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, 0)

# Prepare PSD for whitening the Waveform
delta_f = sample_rate / float(len(hp1))
flen = int(sample_rate / (2 * delta_f)) + 1
psd_hp1 = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, 0)

# Fix the first and last bins to avoid NaNs and 0 Division
psd_td[0] = psd_td[1]
psd_td[len(psd_td) - 1] = psd_td[len(psd_td) - 2]

psd_hp1[0] = psd_hp1[1]
psd_hp1[len(psd_hp1) - 1] = psd_hp1[len(psd_hp1) - 2]


data_whitened = (ts.to_frequencyseries() / psd_td**0.5).to_timeseries()
hp1_whitened = (hp1.to_frequencyseries() / psd_hp1**0.5).to_timeseries() * 1E-21

cross_correlation = numpy.zeros([len(data)-len(hp1)])
# Convert Arrays to NumPy Arrays
hp1n = hp1_whitened.numpy()
datan = data_whitened.numpy()

# Slide Template over the waveform and detect correlations
for i in range(len(datan) - len(hp1n)):
    cross_correlation[i] = (hp1n * datan[i:i+len(hp1n)]).sum()

# Plotting
plt.figure()
times = numpy.arange(len(datan) - len(hp1n)) / float(sample_rate)
plt.plot(times, cross_correlation)
plt.plot([waveform_start/float(sample_rate), waveform_start/float(sample_rate)],
           [(min(cross_correlation))*1.1,(max(cross_correlation))*1.1],'r:')
plt.xlabel('Time (s)')
plt.ylabel('Cross-correlation')
plt.show()